# <a id='toc1_'></a>[__Managed to External table conversion script__](#toc0_)
As you know in unity catalog environment Managed tables are not allowed to upgrde from hive metastore. We have to convert all the Managed tables to External inorder to migrate (__[more info](https://learn.microsoft.com/en-us/azure/databricks/data-governance/unity-catalog/migrate)__)
## Required pacakages
> pandas <br>
> PyYAML

In [ ]:
# Install pandas package
pip install pandas

In [ ]:
# Install PyYAML package
pip install PyYAML

In [ ]:
# Import necessary libraries
import os
import yaml
import pandas as pd

# Initialize the YAML file with the required structure
def init(yamlpath):
    """
    Initialize the YAML file with the required structure.
    
    Parameters:
    yamlpath (str): The path to the YAML file to be initialized.
    """
    with open(yamlpath, "w") as fo:
        fo.write("resources:")
        fo.write("\n")
        fo.write(" tables:\n")

# Write data to the YAML file
def write_yaml(new_yaml_data_dict, yamlpath):
    """
    Write data to the YAML file.
    
    Parameters:
    new_yaml_data_dict (dict): The dictionary containing the data to be written to the YAML file.
    yamlpath (str): The path to the YAML file.
    """
    if not os.path.isfile(yamlpath):
        with open(yamlpath, "a") as fo:
            fo.write("\n")

    #the leading spaces and indent=4 are key here!
    sdump = "  " + yaml.dump(
                new_yaml_data_dict
                ,indent=4
                )

    with open(yamlpath, "a") as fo:
        fo.write(sdump)

# Convert CSV data to YAML format
def csvToYaml(csvFile, yamlpath):
    """
    Convert CSV data to YAML format.
    
    Parameters:
    csvFile (str): The path to the CSV file containing the data.
    yamlpath (str): The path to the YAML file where the data will be written.
    """
    df = pd.read_csv(csvFile)
    records = df.to_dict(orient='records')
    rangenum = range(1,len(records)+1)
    for row,i in zip(records,rangenum):
        tname = row["name"]
        new_yaml_data_dict = {
            f"{tname}":{
                'database':row["database"],
                'name':tname,
                'target_schema':row['target_schema'],
                'target_table':tname,
                'is_pii':row["is_pii"],
                't_no':i
            }
        }
        write_yaml(new_yaml_data_dict,yamlpath)
        
#YAML path
yamlpath = "Test.yml"
#CSV path
csvpath = "Tablelist.csv"
init(yamlpath)
csvToYaml(csvpath,yamlpath)
